# Loop 3 Analysis: Fixing Overlap Validation

The submission failed with 'Overlapping trees in group 002'. Need to understand why my strict validation isn't catching what Kaggle's validator catches.

In [ ]:
import pandas as pd
import numpy as np
import math
from decimal import Decimal, getcontext
from shapely.geometry import Polygon
from shapely import affinity
import os

# Set precision for Decimal (matching official kernel)
getcontext().prec = 25
scale_factor = Decimal('1e15')

# Tree vertices (from official kernel)
TX = [0,0.125,0.0625,0.2,0.1,0.35,0.075,0.075,-0.075,-0.075,-0.35,-0.1,-0.2,-0.0625,-0.125]
TY = [0.8,0.5,0.5,0.25,0.25,0,0,-0.2,-0.2,0,0,0.25,0.25,0.5,0.5]

In [ ]:
def make_tree_polygon_official(center_x, center_y, angle):
    """Create tree polygon using official method with Decimal precision"""
    trunk_w = Decimal('0.15')
    trunk_h = Decimal('0.2')
    base_w = Decimal('0.7')
    mid_w = Decimal('0.4')
    top_w = Decimal('0.25')
    tip_y = Decimal('0.8')
    tier_1_y = Decimal('0.5')
    tier_2_y = Decimal('0.25')
    base_y = Decimal('0.0')
    trunk_bottom_y = -trunk_h

    initial_polygon = Polygon(
        [
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ]
    )
    rotated = affinity.rotate(initial_polygon, float(angle), origin=(0, 0))
    translated = affinity.translate(rotated,
                                    xoff=float(Decimal(str(center_x)) * scale_factor),
                                    yoff=float(Decimal(str(center_y)) * scale_factor))
    return translated

In [ ]:
def check_overlaps_official(xs, ys, degs):
    """Check overlaps using official method: intersects but not touches"""
    n = len(xs)
    if n <= 1:
        return True, []
    
    polygons = [make_tree_polygon_official(xs[i], ys[i], degs[i]) for i in range(n)]
    
    overlaps = []
    for i in range(n):
        for j in range(i+1, n):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                overlaps.append((i, j))
    
    return len(overlaps) == 0, overlaps

In [ ]:
# Load the failed submission
failed_path = '/home/code/submission_candidates/candidate_002.csv'
df = pd.read_csv(failed_path)

# Parse values
df['x_val'] = df['x'].str.replace('s', '').astype(float)
df['y_val'] = df['y'].str.replace('s', '').astype(float)
df['deg_val'] = df['deg'].str.replace('s', '').astype(float)
df['N'] = df['id'].str.split('_').str[0].astype(int)

print(f"Loaded {len(df)} rows")
print(f"N values: {df['N'].nunique()}")

# Check N=2 specifically (the failing group)
n2_data = df[df['N'] == 2]
print(f"\nN=2 data:")
print(n2_data)

In [ ]:
# Check N=2 for overlaps using official method
xs = n2_data['x_val'].values
ys = n2_data['y_val'].values
degs = n2_data['deg_val'].values

print(f"N=2 coordinates:")
for i in range(len(xs)):
    print(f"  Tree {i}: x={xs[i]:.12f}, y={ys[i]:.12f}, deg={degs[i]:.12f}")

valid, overlaps = check_overlaps_official(xs, ys, degs)
print(f"\nN=2 valid (official method): {valid}")
print(f"Overlaps: {overlaps}")

In [ ]:
# Compare with baseline N=2
baseline_path = '/home/code/preoptimized/submission.csv'
baseline_df = pd.read_csv(baseline_path)
baseline_df['x_val'] = baseline_df['x'].str.replace('s', '').astype(float)
baseline_df['y_val'] = baseline_df['y'].str.replace('s', '').astype(float)
baseline_df['deg_val'] = baseline_df['deg'].str.replace('s', '').astype(float)
baseline_df['N'] = baseline_df['id'].str.split('_').str[0].astype(int)

baseline_n2 = baseline_df[baseline_df['N'] == 2]
print("Baseline N=2:")
print(baseline_n2)

xs_b = baseline_n2['x_val'].values
ys_b = baseline_n2['y_val'].values
degs_b = baseline_n2['deg_val'].values

valid_b, overlaps_b = check_overlaps_official(xs_b, ys_b, degs_b)
print(f"\nBaseline N=2 valid: {valid_b}")
print(f"Overlaps: {overlaps_b}")

In [ ]:
# Check ALL N values in failed submission for overlaps
print("Checking all N values for overlaps...")
failing_groups = []

for n in range(1, 201):
    n_data = df[df['N'] == n]
    xs = n_data['x_val'].values
    ys = n_data['y_val'].values
    degs = n_data['deg_val'].values
    
    valid, overlaps = check_overlaps_official(xs, ys, degs)
    if not valid:
        failing_groups.append((n, len(overlaps)))

print(f"\nFailing groups: {len(failing_groups)}")
for n, num_overlaps in failing_groups[:20]:
    print(f"  N={n}: {num_overlaps} overlaps")